# GPT 고객 서비스 챗봇 만들기

In [1]:
# 터미널에서 실행시
# python -m venv venv
# source venv/bin/activate  # 윈도우는 venv\Scripts\activate
# pip install -r requirements.txt
# python app.py

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, jsonify, request, render_template
app = Flask(__name__)

In [2]:
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json.get("message")

    #  OpenAI 모델에 요청
    response = client.chat.completions.create(
        model = "gpt-4o-mini", 
        max_tokens= 300,
        messages = [ {"role":"user", "content":user_message} ],
        n = 1,
        stop = None,
        temperature = 0.7,
    )    

    bot_response = response.choices[0].message.content
    return jsonify({"response": bot_response})
   
           
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


## 질문한 내용을 Context로 계속 유지하면서 질문하기

In [ ]:
# 소스 코드를 이어서 실행할 경우 오류나므로 "Restart Kernel"을 수행하고 실행한다

@app.route('/')
def home():
    return render_template('index.html')

# 대화 히스토리를 저장할 변수
conversation_history = []

@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json.get("message")
    conversation_history.append({"role":"user","content":user_message})
    print(conversation_history)

    # OpenAI 모델에 요청
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = conversation_history,
        max_tokens = 300,
        n = 1,
        stop = None,
        temperature = 0.2,
    )

    bot_response = response.choices[0].message.content
    return jsonify({"response": bot_response})
        
if __name__ == '__main__':
    app.run(debug=False)    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Sep/2025 22:45:13] "GET / HTTP/1.1" 200 -


[{'role': 'user', 'content': '난 홍길동이야'}]


127.0.0.1 - - [25/Sep/2025 22:45:26] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '난 홍길동이야'}, {'role': 'user', 'content': '내이름이 뭐지?'}]


127.0.0.1 - - [25/Sep/2025 22:45:37] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '난 홍길동이야'}, {'role': 'user', 'content': '내이름이 뭐지?'}, {'role': 'user', 'content': '하늘이 파란이유는?'}]


127.0.0.1 - - [25/Sep/2025 22:45:56] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '난 홍길동이야'}, {'role': 'user', 'content': '내이름이 뭐지?'}, {'role': 'user', 'content': '하늘이 파란이유는?'}, {'role': 'user', 'content': '노란이유는?'}]


127.0.0.1 - - [25/Sep/2025 22:46:07] "POST /get_response HTTP/1.1" 200 -
